# Load Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import json
import csv

import pandas as pd
import numpy as np

from tqdm import tqdm
from collections import Counter
from pprint import pprint

from functions.functions_features import split_audio, mean_variance_normalize, process_row, pad_array, process_CNN_row

# Load data

In [2]:
df_all = pd.read_csv('Results/Data/data_all.csv')
df_all

,dataset,filepath,filename,age,gender,label,status,prob
0,coswara,Dataset/Coswara-Data/Extracted_data/20200424/i...,iV3Db6t1T8b7c5HQY2TwxIhjbzD3_cough-shallow,28.0,male,1,healthy,NaN
1,coswara,Dataset/Coswara-Data/Extracted_data/20200424/i...,iV3Db6t1T8b7c5HQY2TwxIhjbzD3_cough-heavy,28.0,male,1,healthy,NaN
2,coswara,Dataset/Coswara-Data/Extracted_data/20200424/i...,iV3Db6t1T8b7c5HQY2TwxIhjbzD3_breathing-shallow,28.0,male,0,healthy,NaN
3,coswara,Dataset/Coswara-Data/Extracted_data/20200424/i...,iV3Db6t1T8b7c5HQY2TwxIhjbzD3_vowel-a,28.0,male,0,healthy,NaN
4,coswara,Dataset/Coswara-Data/Extracted_data/20200424/i...,iV3Db6t1T8b7c5HQY2TwxIhjbzD3_vowel-o,28.0,male,0,healthy,NaN
...,...,...,...,...,...,...,...,...
72335,virufy,Dataset/virufy-data/clinical/segmented/neg/neg...,neg-0422-097-cough-m-37-4.wav,37.0,male,1,negative,NaN
72336,virufy,Dataset/virufy-data/clinical/segmented/neg/neg...,neg-0422-097-cough-m-37-1.wav,37.0,male,1,negative,NaN
72337,virufy,Dataset/virufy-data/clinical/segmented/neg/neg...,neg-0422-098-cough-f-24-5.wav,24.0,female,1,negative,NaN
72338,virufy,Dataset/virufy-data/clinical/segmented/neg/neg...,neg-0422-098-cough-f-24-0.wav,24.0,female,1,negative,NaN


In [3]:
# Get columns
columns = [
    'dataset', 'filepath', 'filename', 
    'age', 'gender', 'label', 'status', 'duration', 
    'duration_segment', 'sample_frequency', 'mean_amplitude',
    'segment_shape',
    ]
    
print(columns)

['dataset', 'filepath', 'filename', 'age', 'gender', 'label', 'status', 'duration', 'duration_segment', 'sample_frequency', 'mean_amplitude', 'segment_shape']


In [4]:
list_dataset_name = [
    'coswara', 
    'coughvid', 
    'esc50', 
    'fsdkaggle', 
    'virufy',
    ]

overlap=0 # To avoid overfitting when doing kfold

In [5]:
if not os.path.exists(f'Results/Features_CNN'):
    os.makedirs(f'Results/Features_CNN')

for segment_length in [1, 5, 10]:
    for dataset_name in list_dataset_name:
        print(f'\n{dataset_name} {segment_length}')
        
        df_all = pd.read_csv(f'Results/Data/data_summary_{dataset_name}.csv')
        df_all = df_all.sample(frac=1).groupby('label').head(1000).reset_index(drop=True)
        
        results_all = []
 
        path_save = f'Results/Features_CNN/data_{dataset_name}_features_{segment_length}s_{overlap}.csv'

        if os.path.exists(path_save) == False:
        # if True:
            for i in tqdm(range(len(df_all))):
                results, output_shape = process_CNN_row(i, df_all, segment_length, overlap)
                results_all.extend(results)
                
            print(output_shape)
            dimension_dictionary = {
                1: 22,
                5: 27,
                10: 27,
            }
            output_shape = dimension_dictionary[segment_length] * 128
            columns_additional = list(range(output_shape))
            results_all = pd.DataFrame(results_all, columns=columns + columns_additional)
            results_all.to_csv(path_save, index=False)
            print(results_all.shape)

coswara 1


100%|██████████| 2000/2000 [01:44<00:00, 19.10it/s]


2816
(17759, 2828)
coughvid 1


100%|██████████| 2000/2000 [01:40<00:00, 19.97it/s]


2816
(16693, 2828)
esc50 1


...

3456
(1040, 3468)
fsdkaggle 10


100%|██████████| 1273/1273 [00:17<00:00, 70.90it/s]


3456
(1625, 3468)
virufy 10


100%|██████████| 121/121 [00:01<00:00, 107.71it/s]


3456
(121, 3468)
